In [ ]:
"""
@Description: Use YOLO8 to Instance Segmentation
@Author: Ken Zh0ng
@date: 2024-06-04
"""
from ultralytics import YOLO
import torch
torch.cuda.empty_cache()

## TRAIN

In [ ]:
# n 32
model = YOLO("yolov8n-seg.pt")
model.info()
results = model.train(data="dataset/yolo8_seg/data.yaml", batch=32, epochs=100, imgsz=640, device=0, verbose=True, plots=True)

In [ ]:
# x 12
model = YOLO("yolov8x-seg.pt")
model.info()
results = model.train(data="dataset/yolo8_seg/data.yaml", batch=10, epochs=100, imgsz=640, device=0, verbose=True, plots=True)

## VAL

In [ ]:
# n 32
model = YOLO("runs/segment/train/weights/best.pt")
metrics = model.val(conf=0.65)

In [ ]:
# x 10
model = YOLO("runs/segment/train2/weights/best.pt")
metrics = model.val(conf=0.65)

In [ ]:
# see one res
import os
testset_path = "dataset/yolo8_seg/test/"
model = YOLO("runs/segment/train2/weights/best.pt")
res = model(os.path.join(testset_path, "images", "800.jpg"))
# Process results list
for result in res:
    masks = result.masks  # Masks object for segmentation masks outputs
    
    print("xxxxx")
    print(type(masks))
    print(masks.xyn)
    print(type(masks.xyn))
    print(len(masks.xyn))
    print(type(masks.xyn[0]))
    print(len(masks.xyn[0]))
    print("yyyyyyyy")
    result.save(filename="result.jpg")  # save to disk

## CAL IOU

In [1]:
# IOU test
from ultralytics import YOLO
import torch
torch.cuda.empty_cache()

import os
import cv2
import numpy as np
from tqdm import tqdm

torch.cuda.empty_cache()

testimgs_path = "dataset/yolo8_seg/test/images"
testlabels_path = "dataset/yolo8_seg/test/raw_labels"

model = YOLO("runs/segment/train/weights/best.pt")

avg_IOU = 0
testset_size = len(os.listdir(testimgs_path))
print(testset_size)

total_tests = os.listdir(testimgs_path)
bar = tqdm(total_tests)
for file_n in bar:
    if file_n in ("990.jpg") :
        continue
    # if file_n in ("869.jpg", "907.jpg", "991.jpg", "990.jpg") :
    #     continue
    reses = model.predict(os.path.join(testimgs_path, file_n))
    
    assert len(reses) == 1
    res = reses[0]
    if res:
        pred_masks = res.masks.xy
    else:
        continue # no res
    
    if pred_masks is None:
        continue
    
    with open(os.path.join(testlabels_path, file_n.replace(".jpg", ".txt")), "r") as f:
        gt_masks = []
        for line in f.readlines():
            line = line.split(" ")
            assert len(line) % 2 == 0
            xl = [float(x) for x in line[::2]]
            yl = [float(y) for y in line[1::2]]
            gt_masks.append(np.array([xl, yl], dtype=np.float32).T)
    
    # assert len(pred_masks) == len(gt_masks)
    
    # cal ious MAT
    ious = np.zeros((len(pred_masks), len(gt_masks)))
    for i, pred_mask in enumerate(pred_masks):
        for j, gt_mask in enumerate(gt_masks):
            intersection_area = cv2.intersectConvexConvex(pred_mask, gt_mask)[0]
            union_area = cv2.contourArea(pred_mask) + cv2.contourArea(gt_mask) - intersection_area
            if union_area<=0:
                ious[i, j] = 0
                continue
            ious[i, j] = intersection_area / union_area
    
    cur_avg_iou = 0
    for i in range(len(pred_masks)):
        cur_avg_iou += ious[i, np.argmax(ious[i])]
    cur_avg_iou /= len(pred_masks)
    
    avg_IOU += cur_avg_iou
IOU = avg_IOU / (testset_size - 3)
print(f"IOU: {IOU}, for model: {model.model_name}")
        

/home/zhongken/miniconda3/envs/yolo8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


200


  0%|          | 0/200 [00:00<?, ?it/s]


image 1/1 /home/zhongken/ai_cross_application/fn/dataset/yolo8_seg/test/images/800.jpg: 576x640 1 polyp, 7.5ms
Speed: 5.9ms preprocess, 7.5ms inference, 2.4ms postprocess per image at shape (1, 3, 576, 640)


  0%|          | 1/200 [00:03<10:04,  3.04s/it]

...


image 1/1 /home/zhongken/ai_cross_application/fn/dataset/yolo8_seg/test/images/988.jpg: 576x640 1 polyp, 6.8ms
Speed: 2.6ms preprocess, 6.8ms inference, 1.4ms postprocess per image at shape (1, 3, 576, 640)

image 1/1 /home/zhongken/ai_cross_application/fn/dataset/yolo8_seg/test/images/989.jpg: 576x640 1 polyp, 6.7ms
Speed: 2.5ms preprocess, 6.7ms inference, 1.6ms postprocess per image at shape (1, 3, 576, 640)

image 1/1 /home/zhongken/ai_cross_application/fn/dataset/yolo8_seg/test/images/991.jpg: 512x640 1 polyp, 7.0ms
Speed: 2.7ms preprocess, 7.0ms inference, 1.4ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /home/zhongken/ai_cross_application/fn/dataset/yolo8_seg/test/images/992.jpg: 544x640 1 polyp, 6.9ms
Speed: 2.4ms preprocess, 6.9ms inference, 1.5ms postprocess per image at shape (1, 3, 544, 640)

image 1/1 /home/zhongken/ai_cross_application/fn/dataset/yolo8_seg/test/images/993.jpg: 512x640 1 polyp, 7.0ms
Speed: 2.2ms preprocess, 7.0ms inference, 1.4ms postproc

 97%|█████████▋| 194/200 [00:06<00:00, 50.54it/s]


image 1/1 /home/zhongken/ai_cross_application/fn/dataset/yolo8_seg/test/images/994.jpg: 512x640 1 polyp, 6.3ms
Speed: 2.2ms preprocess, 6.3ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /home/zhongken/ai_cross_application/fn/dataset/yolo8_seg/test/images/995.jpg: 576x640 1 polyp, 7.2ms
Speed: 2.7ms preprocess, 7.2ms inference, 1.5ms postprocess per image at shape (1, 3, 576, 640)

image 1/1 /home/zhongken/ai_cross_application/fn/dataset/yolo8_seg/test/images/996.jpg: 544x640 1 polyp, 7.3ms
Speed: 2.5ms preprocess, 7.3ms inference, 1.5ms postprocess per image at shape (1, 3, 544, 640)

image 1/1 /home/zhongken/ai_cross_application/fn/dataset/yolo8_seg/test/images/997.jpg: 576x640 1 polyp, 7.1ms
Speed: 2.4ms preprocess, 7.1ms inference, 1.3ms postprocess per image at shape (1, 3, 576, 640)

image 1/1 /home/zhongken/ai_cross_application/fn/dataset/yolo8_seg/test/images/998.jpg: 576x640 1 polyp, 6.7ms
Speed: 2.5ms preprocess, 6.7ms inference, 1.4ms postproc

100%|██████████| 200/200 [00:06<00:00, 28.60it/s]

IOU: 0.6939754100758398, for model: runs/segment/train/weights/best.pt


0-50: 0.7349831990629428

In [2]:
# IOU test
from ultralytics import YOLO
import torch
torch.cuda.empty_cache()

import os
import cv2
import numpy as np
from tqdm import tqdm

torch.cuda.empty_cache()
testimgs_path = "dataset/yolo8_seg/test/images"
testlabels_path = "dataset/yolo8_seg/test/raw_labels"

model = YOLO("runs/segment/train2/weights/best.pt")

file_n = "869.jpg"

reses = model.predict(os.path.join(testimgs_path, file_n))

assert len(reses) == 1
res = reses[0]
if res:
    pred_masks = res.masks.xy
else:
    print("no res")

with open(os.path.join(testlabels_path, file_n.replace(".jpg", ".txt")), "r") as f:
    gt_masks = []
    for line in f.readlines():
        line = line.split(" ")
        assert len(line) % 2 == 0
        xl = [float(x) for x in line[::2]]
        yl = [float(y) for y in line[1::2]]
        gt_masks.append(np.array([xl, yl], dtype=np.float32).T)
        
print(len(pred_masks))
print(len(gt_masks))
intersection_area, _ = cv2.intersectConvexConvex(pred_masks[0], gt_masks[0])
print(intersection_area)
print(pred_masks[1])
print(gt_masks[0])
# intersection_area = cv2.intersectConvexConvex(pred_masks[1], gt_masks[0])[0]
# print(intersection_area)
# cal ious MAT
# ious = np.zeros((len(pred_masks), len(gt_masks)))
# for i, pred_mask in enumerate(pred_masks):
#     for j, gt_mask in enumerate(gt_masks):
#         intersection_area = cv2.intersectConvexConvex(pred_mask, gt_mask)[0]
        # union_area = cv2.contourArea(pred_mask) + cv2.contourArea(gt_mask) - intersection_area
        # assert union_area > 0
        # ious[i, j] = intersection_area / union_area


image 1/1 /home/zhongken/ai_cross_application/fn/dataset/yolo8_seg/test/images/869.jpg: 576x640 2 polyps, 36.5ms
Speed: 3.1ms preprocess, 36.5ms inference, 1.5ms postprocess per image at shape (1, 3, 576, 640)
2
1
95378.65625
[[     215.76      66.737]
 [     214.78      67.709]
 [     191.46      67.709]
 [     191.46      86.175]
 [     190.49      87.147]
 [     190.49      213.49]
 [     191.46      214.46]
 [     191.46      225.15]
 [     192.43      226.12]
 [     192.43      228.07]
 [     195.35      228.07]
 [     196.32      229.04]
 [     197.29      229.04]
 [     202.15       233.9]
 [     202.15      234.87]
 [     205.07      237.79]
 [     205.07      238.76]
 [     206.04      239.73]
 [     206.04       240.7]
 [     207.98      242.65]
 [     207.98      245.56]
 [     207.01      246.53]
 [     207.01      247.51]
 [     206.04      248.48]
 [     206.04      250.42]
 [     204.09      252.37]
 [     204.09      253.34]
 [     203.12      254.31]
 [     202.15    